In [1]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from PIL import Image
import skimage
from skimage import measure
%matplotlib inline

Using TensorFlow backend.


In [6]:
#path = '/home/grads/n/narendra5/Desktop/Programs/LER_machine_learning/'
path = r"C:\\Users\\naren\\Documents\\LER\\"

#model = load_model(path + 'models/' + 'EDGEnet_run_epoch_4.h5')

model = load_model(path + 'LER_machine_learning\\models\\' + 'Linenet_image3_round_L2_epoch_2.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 64)      36928     
__________

C:\Users\naren\Anaconda3\envs\old_tf\lib\site-packages\keras\engine\saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [7]:
np.random.seed(21)
sigma = 1.0
alpha = 0.3
Xi = 22
width = 30
space = 60
noise = 4
shift = math.floor(-25 + (width + space/2 + Xi + alpha*10 + sigma*10)%16) 

original_file = path + 'original_images3\\oim_' + "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '_' + str(-shift) + '.tiff'
noisy_file = path + 'noisy_images3\\nim_' + "{0:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '_' + str(-shift) + '_' + str(noise) + '.tiff'
linescan_file = path + 'linescans\\linescan_' + "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '.txt'


im = np.array(Image.open(original_file))
im = im/256
imnoisy = np.array(Image.open(noisy_file))
imnoisy = (imnoisy)/256

#imnoisy = imnoisy + np.random.normal(0, 0.05 * (256/255), imnoisy.shape)

imnoisy = imnoisy.reshape(1,1024,256,1)

predict = np.zeros([1,1024,256,2])
for k in range(4):
    predict[:,k*256:(k+1)*256, 0:256, :] = model.predict(imnoisy[:,k*256:(k+1)*256, 0:256, :])
    #print(model.predict(imnoisy[:,k*256:(k+1)*256, 0:256, :]).shape)

predict = predict.reshape(1024,256,2)
imnoisy = imnoisy.reshape(1024,256)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\\\Users\\\\naren\\\\Documents\\\\LER\\\\original_images3\\oim_1e-09_0.3_2.2e-08_30_60_10.tiff'

In [ ]:
linescan = []
with open(linescan_file,'r') as f:
    for i,line in enumerate(f):
        if i < 8192:
            a, b = line.split(',')
            linescan.append(float(b))
        else:
            break
print(len(linescan))

linescan = np.array(linescan)
linescan = linescan + shift
linescan = linescan.round().astype(int)
edgeimage = np.zeros((1024,256))

if width == 30:                             #number of edges or peaks in a line
    nedges = 6
else:
    nedges = 8


original_edges = np.zeros((nedges, 1024))
for k in range(nedges):                                  #for k edges
    if k%2 == 0:                                    #keep even edges same 
        edge = linescan[k*1024:(k+1)*1024]
    else:
        edge = np.flip(linescan[k*1024:(k+1)*1024],0)  #flip odd edges
    
    original_edges[k,:] = edge
    for i in range(1024):
        if edge[i] >= 0 and edge[i] <= 255:
            edgeimage[i, edge[i]] = 1
            edgeimage[i, edge[i]] = 1

            
psnr_noisy = measure.compare_psnr(im,imnoisy.astype(float))
psnr_predict = measure.compare_psnr(im,predict[:,:,0].astype(float))
print('PSNR noisy image = ', psnr_noisy)
print('PSNR predicted image = ', psnr_predict)

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(16, 16), sharex=True,
                       sharey=True, subplot_kw={'adjustable': 'box-forced'})

ax[0,0].imshow(im,cmap = 'gray',aspect= 0.5)
ax[0,0].set_title('(a) Original Image', fontsize=18)
ax[0,0].tick_params(labelsize=15)

ax[0,1].imshow(imnoisy, cmap = 'gray',aspect= 0.5)
ax[0,1].set_title('(b) Noisy Image: PSNR = {0:.4g}'.format(psnr_noisy), fontsize=18)
ax[0,1].tick_params(labelsize=15)

ax[0,2].imshow(predict[:,:,0], cmap = 'gray',aspect= 0.5)
ax[0,2].set_title('(c) Predicted Denoised Image \n PSNR = {0:.4g}'.format(psnr_predict), fontsize=18)

ax[1,0].imshow(edgeimage, cmap = 'gray',aspect= 0.5)
ax[1,0].set_title('(c) True Edge Image', fontsize=18)
ax[1,0].tick_params(labelsize=15)

ax[1,1].imshow(predict[:,:,1], cmap = 'gray',aspect= 0.5)
ax[1,1].set_title('(e) Predicted Edge Image', fontsize=18)
ax[1,1].tick_params(labelsize=15)
#ax[1,0].imshow(np.abs(predict[:,:,1] - edgeimage), cmap = 'gray',aspect= 0.5)
#ax[1,0].set_title('EDGE Difference Image')

In [ ]:
# code for 3d plot and finding peaks

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

fig = plt.figure(figsize=(16, 10))
ax = fig.add_subplot(111,projection='3d')
ax.view_init(elev=45., azim=10)

x, y = np.mgrid[0:1024, 0:256]
surf = Axes3D.plot_surface(ax, x, y, predict[:,:,1], cmap=cm.gray)
plt.show()

In [ ]:
from scipy.signal import find_peaks    # To get maxima

N = 256
#print(find_peaks(predict[N,:,1], height = 0.1, distance = 3))
plt.figure(figsize = (10,6))
plt.plot(predict[N,:100,1], '-o', label = 'Predicted Edge image')                         # plot one row of edge image
plt.plot(edgeimage[N,:100], '-o', label = 'True Edge image')
plt.xlabel('pixels', fontsize = 16)
plt.ylabel('Intensity', fontsize = 16)
plt.legend(fontsize=14)
plt.show()

plt.figure(figsize = (10,6))
plt.plot(im[N,:100], '-o', label = 'Original image')
plt.plot(predict[N,:100,0], '-o', label = 'Predicted denoised image')
#plt.plot(imnoisy[N,:100], '-o', label = 'Noisy image')
plt.xlabel('pixels', fontsize = 16)
plt.ylabel('Intensity', fontsize = 16)
plt.legend(fontsize=14)
plt.show()

plt.figure(figsize = (10,6))
plt.plot(imnoisy[N,:100], '-o', label = 'Noisy image')
plt.xlabel('pixels', fontsize = 16)
plt.ylabel('Intensity', fontsize = 16)
plt.legend(fontsize=14)
plt.show()

In [ ]:

predicted_edges = np.zeros((nedges, 1024))
mean_edge_pos = np.zeros(8)
dist = 10         # initial minimum distance between the peaks
over = 0          # flag peak finishing peak search

while(not over):
    mean_edge_pos = np.zeros(8)
    for i in range(1024):
        peaks, _ = find_peaks(predict[i,:,1], height = 0.11, distance = dist)
    
        mean_edge_pos = predicted_edges[:,:i].mean(axis=1)       # mean edge position
        if len(peaks) < nedges:
            dist = dist - 1
            break
        elif len(peaks) == nedges or i == 0:
            for k in range(nedges):
                predicted_edges[k,i] = peaks[k]
        else:
            for k in range(nedges):
                min_index = np.argmin(np.abs(peaks - mean_edge_pos[k]))
                predicted_edges[k,i] = peaks[min_index]
        if i == 1023:
            print (dist)
            over = 1

In [ ]:
print(original_edges.std(axis = 1)/2)
print(predicted_edges.std(axis = 1)/2)


print("LER: ",(original_edges.std(axis = 1)/2).mean(), (predicted_edges.std(axis = 1)/2).mean())


print("MABSE:", np.abs(original_edges - predicted_edges).sum(axis=1).sum()/(nedges*1024))

original_LWR = np.zeros(nedges//2)
predicted_LWR = np.zeros(nedges//2)

for i in range(nedges//2):
    original_LWR[i] = (original_edges[i+1,:] - original_edges[i,:]).std()/2
    predicted_LWR[i] = (predicted_edges[i+1,:] - predicted_edges[i,:]).std()/2

print("LWR: ", original_LWR.mean(), predicted_LWR.mean())
print("PSNR: ", psnr_noisy, psnr_predict)
    
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 8), sharex=True,
                       sharey=False, subplot_kw={'adjustable': 'box-forced'})

ax[0].plot(original_edges.T)
ax[0].set_title('Original linescans')

ax[1].plot(predicted_edges.T)
ax[1].set_title('Predicted linescans')

plt.show()